In [2]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

## Group Assignment
### Team Number: 06
### Team Member Names: Shayan Jalali, Krish Suryavanshi, Paul Reddy
### Team Strategy Chosen: Market Meat

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.

In [3]:
# Krish's code

csv_file = "Tickers_Example.csv"
tickers = pd.read_csv(csv_file, header=None, names=["Ticker"])

# Include only valid US and Canadian companies that have listed stocks in these markets.

start_date = "2024-10-01"
end_date = "2025-10-01" # TO INCLUDE SEPT 30TH

valid_tickers = []

for ticker in tickers["Ticker"]:

    if "." not in ticker or ticker.endswith(".TO"):

        hist = yf.Ticker(ticker).history(start=start_date, end=end_date)
        
        if not hist.empty:
            valid_tickers.append(ticker)

# Exclude any stock with average daily volume < 5,000 shares between Oct 1, 2024 and Sep 30, 2025 (drop months with < 18 trading days).

volume_cutoff = 5000
minimum_trading_days = 18
final_tickers = []

for valid_ticker in valid_tickers:

    hist = yf.Ticker(valid_ticker).history(start=start_date, end=end_date)
    months = list(hist.index.strftime("%Y-%m"))

    unique_months = []
    for month in months:
        if month not in unique_months:
            unique_months.append(month)
    
    good_months = []
    for unique_month in unique_months:
        if months.count(unique_month) >= minimum_trading_days:
            good_months.append(unique_month)
    
    mask = hist.index.strftime("%Y-%m").isin(good_months)
    filtered_hist = hist[mask]

    avg_volume = filtered_hist["Volume"].mean()
    if avg_volume >= volume_cutoff:
        final_tickers.append(valid_ticker)

# create closing prices DataFrame for the filtered tickers

close_prices = pd.DataFrame()
for final_ticker in final_tickers:
    hist = yf.Ticker(final_ticker).history(start=start_date, end=end_date)
    close_prices[final_ticker] = hist["Close"]

close_prices.index = close_prices.index.strftime("%Y-%m-%d")

close_prices

$AGN: possibly delisted; no timezone found
$CELG: possibly delisted; no timezone found
$MON: possibly delisted; no timezone found
$RTN: possibly delisted; no timezone found


,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BB.TO,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2024-10-01,225.162079,188.798172,110.925461,345.618652,71.177116,185.130005,265.044800,154.220001,38.318245,3.29,...,162.130722,160.253281,105.980003,21.057087,81.035980,193.983353,570.918274,239.846207,125.324356,42.547169
2024-10-02,225.729446,188.663986,111.003593,349.850769,70.912827,184.759995,267.334106,152.889999,38.328014,3.27,...,164.614807,159.121506,106.830002,20.899042,80.413269,195.744202,579.737061,237.697311,125.079842,42.346203
2024-10-03,224.624588,187.350739,109.626320,354.711304,70.805168,181.960007,265.034912,150.520004,38.347557,3.15,...,165.201584,157.817047,105.980003,20.713110,80.941620,194.050705,580.167664,235.040497,122.926392,42.087822
2024-10-04,225.749359,186.238800,110.026802,355.693207,74.436455,186.509995,273.028809,155.000000,39.187778,3.23,...,165.182022,159.409241,111.910004,20.555063,81.620941,195.051422,578.650574,233.311615,123.424774,42.776840
2024-10-07,220.663040,185.586990,110.730095,350.351532,72.077583,180.800003,270.872040,155.910004,39.041233,3.19,...,163.265182,159.121506,110.190002,20.601547,82.017197,193.665848,571.172729,231.367844,123.377762,42.556736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-24,252.309998,218.415421,133.297256,239.080002,77.900002,220.210007,339.619476,215.100006,51.700001,5.93,...,173.550003,203.245789,206.839996,21.969999,107.572151,182.808289,351.809998,230.360001,83.900002,48.657986
2025-09-25,256.869995,216.975800,132.719818,232.559998,77.099998,218.149994,339.320221,213.529999,51.849998,6.47,...,169.679993,202.332642,199.960007,21.820000,108.929314,180.429520,345.559998,232.000000,82.580002,48.509575
2025-09-26,255.460007,219.030991,132.958771,238.970001,77.980003,219.779999,340.816437,221.259995,52.209999,6.89,...,169.199997,202.114273,195.399994,21.830000,109.315651,182.917328,344.079987,235.199997,83.720001,48.915222


In [4]:
# Shayan's code



In [20]:
# Paul's code

#Get the history of the S&P500 and TSX
sp_data = yf.Ticker("^GSPC").history(start=start_date, end=end_date)
tsx_data = yf.Ticker("^GSPTSE").history(start=start_date, end=end_date)
sp_close = sp_data["Close"]
tsx_close = tsx_data["Close"]

#Calculate cumulative total returns 
sp_returns = sp_close.pct_change()
tsx_returns = tsx_close.pct_change()
sp_total_return = sp_returns.cumsum()*100
tsx_total_return = tsx_returns.cumsum()*100

#Average
benchmark_avg = (sp_total_return + tsx_total_return) / 2

#Store them in dataframes
sp_df = pd.DataFrame({"S&P500 TotalReturn": sp_total_return})
tsx_df = pd.DataFrame({"TSX TotalReturn": tsx_total_return})
benchmark_df = pd.DataFrame({"Benchmark Average": benchmark_avg})

benchmark_df.index = benchmark_df.index.strftime("%Y-%m-%d")

print("Benchmark Average:")

print(benchmark_df)

#Calculate cumulative total returns for stocks
stock_returns = close_prices.pct_change()
stock_total_returns = stock_returns.cumsum() * 100


#Merge the returns of the stocks and the benchmark average
merged = pd.merge(stock_total_returns, benchmark_df, on="Date", how="inner")

#Store the average distances between the returns of the stock to the returns of the benchmark average in a distances dataframe
distances_dict = {}
for ticker in close_prices.columns:
    avg_distance = (merged[ticker] - merged["Benchmark Average"]).abs().mean()
    distances_dict[ticker] = avg_distance

distances_df = pd.DataFrame(list(distances_dict.items()), columns=["Ticker", "Average Distance"])
distances_df = distances_df.sort_values(by="Average Distance", ascending=True).reset_index(drop=True)
distances_df



Benchmark Average:
            Benchmark Average
Date                         
2024-10-01                NaN
2024-10-02          -0.060486
2024-10-03          -0.213509
2024-10-04           0.640328
2024-10-07           0.036659
...                       ...
2025-09-24          19.535943
2025-09-25          19.243483
2025-09-26          19.587850
2025-09-29          20.073442
2025-09-30          20.362898

[256 rows x 1 columns]


/var/folders/nh/705y7gxx1zzdg6v10psp6v500000gp/T/ipykernel_3252/2965462564.py:30: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  stock_returns = close_prices.pct_change()


,Ticker,Average Distance
0,RY.TO,1.616308
1,BLK,4.284487
2,USB,4.455742
3,AXP,6.071995
4,AIG,6.400786
5,AAPL,6.909028
6,ABT,7.203753
7,ABBV,7.267383
8,QCOM,7.435333
9,T.TO,7.546130
